# BERT를 이용한 Chatbot 만들기
> 작성자      
```
* 김성현 (bananaband657@gmail.com)  
1기 멘토
김바다 (qkek983@gmail.com)
박상희 (parksanghee0103@gmail.com)  
이정우 (jungwoo.l2.rs@gmail.com)
2기 멘토
박상희 (parksanghee0103@gmail.com)  
이정우 (jungwoo.l2.rs@gmail.com)
이녕우 (leenw2@gmail.com)
박채훈 (qkrcogns2222@gmail.com)
3, 4, 5기 멘토
이녕우 (leenw2@gmail.com)
박채훈 (qkrcogns2222@gmail.com)
```
[CC BY-NC-ND](https://creativecommons.org/licenses/by-nc-nd/2.0/kr/)


###**콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다.




이번 과제의 목표는 BERT를 사용해 간단한 Chatbot을 어떻게 만들지 고민해 봅니다.

아래 실습 이후, 과제를 위한 챗봇 데이터는 다음의 링크에서 받아서 활용해주세요.   
https://github.com/songys/Chatbot_data   

In [1]:
# !pip install transformers

## Pretrained BERT의  [CLS] token을 이용한 Chatbot.
### 사전 준비물
1. Pretrain된 BERT 모델.
2. 질의응답 Dataset.

### 진행 과정
1. 사용자의 질문(query)를 입력 받는다.
2. query를 pretrained BERT의 입력으로 넣어, query 문장에 해당하는 [CLS] token hidden을 얻는다.
3. 사전에 준비된 질의응답 Dataset에 존재하는 모든 질문들을 pretrained BERT의 입력으로 넣어, 질문들에 해당하는 [CLS] token hidden을 얻는다.
4. query의 [CLS] token hidden과 질문들의 [CLS] token hidden간의 코사인 유사도를 구한다.
5. 가장 높은 코사인 유사도를 가진 질문의 답변을 반환시켜준다.
6. 위 과정 반복.

### Pretrained BERT Load
* 공개된 BERT base Multilingual 버전을 사용한다.

In [2]:
import torch
from transformers import AutoModel, AutoTokenizer

In [3]:
MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)
model.parameters

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<bound method Module.parameters of BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropou

### 질의응답 Dataset 예시
* 같은 index가 서로 pair 입니다.

In [4]:
chatbot_Question = ['기차 타고 여행 가고 싶어','꿈이 이루어질까?','내년에는 더 행복해질려고 이렇게 힘든가봅니다', '간만에 휴식 중', '오늘도 힘차게!'] # 질문
chatbot_Answer = ['꿈꾸던 여행이네요.','현실을 꿈처럼 만들어봐요.','더 행복해질 거예요.', '휴식도 필요하죠', '아자아자 화이팅!!'] # 답변
print(chatbot_Question[:])
print(chatbot_Answer[:])

['기차 타고 여행 가고 싶어', '꿈이 이루어질까?', '내년에는 더 행복해질려고 이렇게 힘든가봅니다', '간만에 휴식 중', '오늘도 힘차게!']
['꿈꾸던 여행이네요.', '현실을 꿈처럼 만들어봐요.', '더 행복해질 거예요.', '휴식도 필요하죠', '아자아자 화이팅!!']


### [CLS] token을 얻기 위한 함수

In [5]:
def get_cls_token(sent_A):
    model.eval()
    tokenized_sent = tokenizer(
            sent_A,
            return_tensors="pt",
            truncation=True,
            add_special_tokens=True,
            max_length=128
    )
    with torch.no_grad():# 그라디엔트 계산 비활성화
        outputs = model(    # **tokenized_sent
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent['token_type_ids']
            )
    logits = outputs.last_hidden_state[:,0,:].detach().cpu().numpy()
    return logits

* query [CLS] token hidden 확인

In [6]:
query = '아 여행가고 싶다~'
query_cls_hidden = get_cls_token(query)
print(query_cls_hidden)
print(query_cls_hidden.shape)

[[ 1.10308655e-01 -1.02158241e-01  2.36618429e-01  5.52498028e-02
   3.93725008e-01  2.43569389e-01  1.86354257e-02 -2.02085257e-01
  -9.63343084e-02  3.54375154e-01 -7.72632882e-02  3.26404870e-01
  -1.74165487e-01  6.97083846e-02 -5.34731708e-02 -1.16455123e-01
  -2.62404919e-01  1.22646175e-01  5.06818056e-01 -9.67913717e-02
   1.23077504e-01  7.92989880e-02 -2.98082363e-02  3.16681653e-01
  -1.79104879e-02  2.30732366e-01  3.72240514e-01 -1.27580971e-01
   5.79191744e-01  4.66822684e-01  4.10569489e-01  1.01960227e-01
  -2.56263435e-01  2.49146074e-01  4.32022393e-01  1.58709854e-01
  -1.90560210e+00  1.18138984e-01 -2.43938621e-02 -7.49029964e-02
  -5.16358852e-01  1.96145639e-01  3.76857340e-01  1.58740178e-01
   5.48579872e-01  1.11865234e+00  6.56481743e-01  8.93772840e-02
   1.43516266e+00 -5.34082294e-01  3.55657935e-01 -6.39795303e-01
  -5.98753318e-02 -1.56276703e+00  2.52752930e-01  1.68553427e-01
   3.63083422e-01 -1.66130182e-03  9.51945856e-02 -1.09238923e-01
  -3.28797

* Chatbot 데이터셋의 질문 [CLS] token hidden 확인

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

dataset_cls_hidden = []
for q in chatbot_Question:
    q_cls = get_cls_token(q)
    dataset_cls_hidden.append(q_cls)
dataset_cls_hidden = np.array(dataset_cls_hidden).squeeze(axis=1)
print(dataset_cls_hidden)   # 데이터셋의 질문에 대한 [CLS] 토큰 벡터
print(dataset_cls_hidden.shape)


[[ 0.20152886  0.00450004  0.19752167 ...  0.54929197  0.02755531
   0.21813379]
 [-0.09731738  0.10722584 -0.23476292 ...  0.36495006  0.45496792
   0.31987938]
 [-0.23625132  0.13388452 -0.28571615 ...  0.53231597  0.33486864
   0.3000063 ]
 [ 0.02712058  0.12172298  0.07334885 ...  0.23747033  0.2893384
  -0.30687737]
 [-0.2990096  -0.09007687  0.10507915 ...  0.378813    0.3802543
   0.3048731 ]]
(5, 768)


### 코사인 유사도

In [8]:
cos_sim = cosine_similarity(query_cls_hidden, dataset_cls_hidden)   # 데이터셋의 0번째 질문과 가장 유사하군요!
print(cos_sim)

[[0.85016316 0.77888525 0.73615134 0.7798742  0.7242013 ]]


* Chatbot 데이터 셋중 가장 유사도가 높은 질문 선택 및 답변

In [9]:
top_question = np.argmax(cos_sim)

print('나의 질문: ', query)
print('저장된 답변: ', chatbot_Answer[top_question])

나의 질문:  아 여행가고 싶다~
저장된 답변:  꿈꾸던 여행이네요.
